<a href="https://colab.research.google.com/github/saccordi/InsightPlaces/blob/master/InsightPlaces_tratamento_dos_dados_para_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bem-vindo(a) ao meu notebook!**
Na etapa anterior, foram realizadas as transformações dos dados. Agora é necessário fazer a exploração e tratamento dessa base de dados para efetivamente partir pra criação dos modelos de ML.

Nesse sentido, inicialmente serão selecionadas as features que fazem mais sentido para o projeto e depois será realizado alguns tratamentos na base de dados. Ao final, esses dados serão utilizados para a criação de um modelo de regressão que irá ajudar a imobiliária *InsightPlaces* na precificação dos imóveis.

In [1]:
# instalando o PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=17153c7e679c7e20bf64a27c0709c1975b10c500137400caff36181837cc0233
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# iniciando uma sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Transformacao com Spark") \
    .getOrCreate()

spark

In [3]:
# obtendo os dados já transformados na fase anterior
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip' && unzip semana-2.zip -d dados/

--2023-11-29 23:56:09--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.116.233, 3.5.11.213, 3.5.0.185, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.116.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2763652 (2.6M) [application/zip]
Saving to: ‘semana-2.zip’

semana-2.zip        100%[===================>]   2.63M  4.37MB/s    in 0.6s    

2023-11-29 23:56:11 (4.37 MB/s) - ‘semana-2.zip’ saved [2763652/2763652]

Archive:  semana-2.zip
   creating: dados/dataset_transformado_parquet/
  inflating: dados/dataset_transformado_parquet/_SUCCESS  
  inflating: dados/dataset_transformado_parquet/._SUCCESS.crc  
  inflating: dados/dataset_transformado_parquet/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet  
  inflating: dados/dataset_transformado_parquet/.part-000

In [4]:
# fazendo a leitura dos dados e mostrando as primeiras linhas
dados = spark.read.parquet("dados/dataset_transformado_parquet")

## Analisando o conjunto de dados

In [5]:
# mostrando as 10 primeiras linhas do conjunto de dados
dados.show(10)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  NULL|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|NULL|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [6]:
# analisando a estrutura da base de dados
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



Percebe-se, com essa primeira analise, que o dados estão transformados de acordo com a fase anterior.

## Seleção de features

Algumas colunas possuem apenas um único valor devido aos filtros que foram realizados na base de dados durante a fase de transformação dos dados. Dessa forma, essas colunas serão excluídas, pois como possuem apenas um valor não serão relevantes para o modelo.

In [7]:
# verificando os registros distintos na coluna tipo
dados.select("tipo").distinct().show(5)

+-----+
| tipo|
+-----+
|Venda|
+-----+



In [8]:
# verificando os registros distintos na coluna anuncio
dados.select("tipo_anuncio").distinct().show(5)

+------------+
|tipo_anuncio|
+------------+
|       Usado|
+------------+



In [9]:
# verificando os registros distintos na coluna tipo_unidade
dados.select("tipo_unidade").distinct().show(5)

+------------+
|tipo_unidade|
+------------+
| Apartamento|
+------------+



In [10]:
# verificando os registros distintos na coluna tipo_uso
dados.select("tipo_uso").distinct().show(5)

+-----------+
|   tipo_uso|
+-----------+
|Residencial|
+-----------+



In [11]:
# verificando os registros distintos nas colunas area_total e area_util
dados.select("area_total", "area_util").show(5)

+----------+---------+
|area_total|area_util|
+----------+---------+
|        43|       43|
|        44|       44|
|      NULL|       55|
|      NULL|       55|
|      NULL|       50|
+----------+---------+
only showing top 5 rows



As colunas `area_total` e `area_util` possuem vários registros distintos e nulos. Sendo assim, será feita uma analise mais profunda.

In [12]:
# importando o módulo functions
from pyspark.sql import functions as f

# criando um DataFrame sem valores nulos das colunas area_util e area_total
dados_without_na = dados\
    .select('area_util', 'area_total')\
    .na\
    .drop()

# criando variáves com analises mais detalhadas
quantidade_total = dados_without_na.count()
quantidade_iguais = dados_without_na.filter(f.col('area_util') == f.col('area_total')).count()
quantidade_distintas = quantidade_total - quantidade_iguais

# mostrando as informações
print(f"Quantidade de linhas: {quantidade_total}")
print(f"Quantidade de linhas com area_util == area_total: {quantidade_iguais}")
print(f"Quantidade de linhas diferentes: {quantidade_distintas}")

Quantidade de linhas: 57368
Quantidade de linhas com area_util == area_total: 55384
Quantidade de linhas diferentes: 1984


Com essa analise mais profunda, percebe-se que existem 57368 linhas e, entre elas, 55384 possuem os mesmos valores tanto na coluna `area_util` quanto na coluna `area_total`. Ou seja, a maioria das informações dessas colunas são as mesmas e,por isso, não é necessário manter as duas colunas.

Para decidir qual das colunas manter, será feita uma alanlise da quantidade de dados nulos em cada uma delas.

In [13]:
# mostrando quantos registros nulos têm em cada uma das colunas
colunas = ["area_total", "area_util"]

dados\
    .select([f.count(f.when(dados[c].isNull(), True)).alias(c) for c in colunas])\
    .show()

+----------+---------+
|area_total|area_util|
+----------+---------+
|      9194|        0|
+----------+---------+



O resultado acima mostra que a coluna `area_total` têm 9194 registros nulos, ao passo que a coluna `area_total` não têm registros nulos. Sendo assim, a coluna `area_total` será removida da base de dados. Além dela, as outras colunas que possuem apenas um único valor também sersão removidas.

In [14]:
# visualizando o novo Dataframe
dados.\
    drop("area_total", "tipo_anuncio", "tipo_unidade", "tipo_uso", "tipo")\
    .show(5)

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|      bairro|        zona|condominio|iptu| valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|  NULL|   1|    Realengo|  Zona Oeste|       285|NULL| 22999|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                  []|      2|     0|   0|       Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|       55|        1|                  []|      2|     0|   1|      Cosmos|  Zona Oeste|      NULL|NULL|115000|
|a6ab01ae-3d40-40e...|    2|       55|        1|                  []|      2|     0|   0|Tomás Coelho|  Zona Norte|      NULL|NULL|115000|
|1d8ff7a2-16bd-4e3...|    0

In [15]:
# aplicando as mudanças
dataset = dados.\
                drop("area_total", "tipo_anuncio", "tipo_unidade", "tipo_uso", "tipo")

dataset.show(5)

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|      bairro|        zona|condominio|iptu| valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|  NULL|   1|    Realengo|  Zona Oeste|       285|NULL| 22999|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                  []|      2|     0|   0|       Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|       55|        1|                  []|      2|     0|   1|      Cosmos|  Zona Oeste|      NULL|NULL|115000|
|a6ab01ae-3d40-40e...|    2|       55|        1|                  []|      2|     0|   0|Tomás Coelho|  Zona Norte|      NULL|NULL|115000|
|1d8ff7a2-16bd-4e3...|    0

## Convertendo os tipos das colunas

Ao analisar a estrutura dos dados, percebe-se que os tipos dos dados estão incorretos e, por isso, será necessário convertê-los.
As colunas `andar`, `banheiros`, `suites` e `quartos`serão convertidas para inteiro. Além disso também pode ser necessário converter as colunas `area_util`, `condominio`, `iptu` e `iptu` para o tipo double.



In [16]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [17]:
# importando o módulo para altear os tipos dos dados
from pyspark.sql.types import IntegerType, DoubleType

In [18]:
# conultando as mudança
dataset\
    .withColumn('andar', dados['andar'].cast(IntegerType()))\
    .withColumn('area_util', dados['area_util'].cast(DoubleType()))\
    .withColumn('banheiros', dados['banheiros'].cast(IntegerType()))\
    .withColumn('quartos', dados['quartos'].cast(IntegerType()))\
    .withColumn('suites', dados['suites'].cast(IntegerType()))\
    .withColumn('condominio', dados['condominio'].cast(DoubleType()))\
    .withColumn('vaga', dados['vaga'].cast(IntegerType()))\
    .withColumn('iptu', dados['iptu'].cast(DoubleType()))\
    .withColumn('valor', dados['valor'].cast(DoubleType()))\
    .printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



In [19]:
# aplicando as mudanças ao Dataset
dataset = dataset\
    .withColumn('andar', dados['andar'].cast(IntegerType()))\
    .withColumn('area_util', dados['area_util'].cast(IntegerType()))\
    .withColumn('banheiros', dados['banheiros'].cast(IntegerType()))\
    .withColumn('quartos', dados['quartos'].cast(IntegerType()))\
    .withColumn('suites', dados['suites'].cast(DoubleType()))\
    .withColumn('condominio', dados['condominio'].cast(DoubleType()))\
    .withColumn('vaga', dados['vaga'].cast(DoubleType()))\
    .withColumn('iptu', dados['iptu'].cast(DoubleType()))\
    .withColumn('valor', dados['valor'].cast(DoubleType()))

## Tratando a coluna "caracteristicas"

A coluna `caracteristicas` possui listas de strings e, algumas dessas listas, estão sem elementos. Essas listas serão transformadas em valores nulos.

In [20]:
# visualizando os elementos únicos das listas dessa coluna
dataset.select(f.explode("caracteristicas")).distinct().collect()

[Row(col='Condomínio fechado'),
 Row(col='Playground'),
 Row(col='Portão eletrônico'),
 Row(col='Piscina'),
 Row(col='Animais permitidos'),
 Row(col='Portaria 24h'),
 Row(col='Elevador'),
 Row(col='Academia'),
 Row(col='Salão de festas'),
 Row(col='Churrasqueira')]

In [21]:
# substituindo os valores vazios da coluna 'caracteristicas'por nulo
dataset = dataset.withColumn('caracteristicas', f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col('caracteristicas')))
dataset.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|  iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|  NULL| 1.0|            Realengo|  Zona Oeste|     285.0|  NULL| 22999.0|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                NULL|      2|   0.0| 0.0|               Irajá|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|       55|        1|                NULL|      2|   0.0| 1.0|              Cosmos|  Zona Oeste|      NULL|  NULL|115000.0|
|a6ab01ae-3d40-40e...|    2|       55|        1|                NULL|      2|   0.0| 0.0|     

## Tratando dados faltantes
Ao analisar a estrutura da base de dados, nota-se que existem diversos dados faltantes. É importante tratar isso pois modelos de Machine Learning não costumam trabalhar corretamente com dados nulos. Tendo isso em mente, os dados nulos das colunas numéricas serão substituídos pelo número 0.


In [22]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



In [23]:
# analisando a quantidade de dados nulos por as coluna
dataset\
    .select([f.count(f.when(dataset[c].isNull(), True)).alias(c) for c in dataset.columns])\
    .show()

+---+-----+---------+---------+---------------+-------+------+----+------+----+----------+----+-----+
| id|andar|area_util|banheiros|caracteristicas|quartos|suites|vaga|bairro|zona|condominio|iptu|valor|
+---+-----+---------+---------+---------------+-------+------+----+------+----+----------+----+-----+
|  0|    0|        0|        0|          12736|      0|  5554|3017|     0|   0|      2371|7199|    0|
+---+-----+---------+---------+---------------+-------+------+----+------+----+----------+----+-----+



In [24]:
# visualizando a transformação
dataset\
    .select('*')\
    .na\
    .fill(0)\
    .show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|  iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|   0.0| 1.0|            Realengo|  Zona Oeste|     285.0|   0.0| 22999.0|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                NULL|      2|   0.0| 0.0|               Irajá|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|       55|        1|                NULL|      2|   0.0| 1.0|              Cosmos|  Zona Oeste|       0.0|   0.0|115000.0|
|a6ab01ae-3d40-40e...|    2|       55|        1|                NULL|      2|   0.0| 0.0|     

In [25]:
# aplicando a transformação
dataset = dataset\
    .select('*')\
    .na\
    .fill(0)

Futuramente será abordado os dados nulos da coluna `caracteristicas`. Nesse momento, é necessário analisar a coluna `zona`. Nessa coluna existe uma string vazia.

In [26]:
# contando os registros por zona
dataset\
    .select('zona')\
    .groupBy('zona')\
    .count()\
    .show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



Pode-se notar que exitem 11 linhas em em uma strig vazia. Elas serão removidas da nossa base de dados.

In [27]:
# removendo a string vazia
dataset = dataset\
    .where(f.col('zona') != '')

In [28]:
# contando os registros por zona
dataset\
    .select('zona')\
    .groupBy('zona')\
    .count()\
    .show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
+------------+-----+



## Preparando os dados para ML

Para que seja possível utilizar os dados em um modelo [Machine Learning do Pyspark](https://), é necessário realizar algumas técnicas como transformação de variáveis categóricas em binárias. Além disso, será feira a vetorização dos dados.




## Variáveis Dummy

Esse processo serve para transformar viriáveis categóricas em binárias.

In [29]:
# visualizando a estrutura dos dadoss
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = false)
 |-- vaga: double (nullable = false)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = false)
 |-- iptu: double (nullable = false)
 |-- valor: double (nullable = false)



In [30]:
# expandindo a coluna "caracteristicas" em múltiplas linhas
dataset.select("*", f.explode("caracteristicas").alias("caracteristica")).show(15, truncate = False)

+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+----+------------+----------+----------+----+--------+------------------+
|id                                  |andar|area_util|banheiros|caracteristicas                                                                                                                |quartos|suites|vaga|bairro      |zona      |condominio|iptu|valor   |caracteristica    |
+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+----+------------+----------+----------+----+--------+------------------+
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43       |1        |[Churrasqueira, Academia, Playground, Salão de festas, Condomínio fechado, Portão eletrônico,

In [31]:
# atribuindo o valor 1 quando a característica está presente
# atribuindo o valor 0 quando a característica está ausente
dataset\
    .select("*", f.explode("caracteristicas").alias("cara"))\
    .groupBy("id")\
    .pivot('cara')\
    .agg(f.lit(1))\
    .na\
    .fill(0)\
    .show(truncate=False)

+------------------------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|id                                  |Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+------------------------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416a-9f84-29091cefe14c|1       |1                 |1            |1                 |1       |1      |1         |1           |1                |1              |
|bfffedfe-99e7-4aed-9a9f-f766225a0f1f|0       |1                 |1            |1                 |1       |0      |1         |0           |1                |1              |
|fcb67af3-5601-415f-9d1f-75280923a4e3|1       |1                 |1            |0                 |0       |1      |1        

In [32]:
# aplicando as transformações a uma variável
caracteristicas = dataset\
    .select("*", f.explode("caracteristicas").alias("cara"))\
    .groupBy("id")\
    .pivot('cara')\
    .agg(f.lit(1))

O mesmo processo feito com a coluna `caracteristicas` será feito com a coluna `zona`. Cada uma das zonas da coluna `zonas` será será transformada em colunas separadas.

In [33]:
# aplicando as transformações a uma variável
zona = dataset\
    .groupBy("id")\
    .pivot('zona')\
    .agg(f.lit(1))

In [34]:
dataset.show(10, truncate=False)

+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+----+-----------------------+------------+----------+------+--------+
|id                                  |andar|area_util|banheiros|caracteristicas                                                                                                                |quartos|suites|vaga|bairro                 |zona        |condominio|iptu  |valor   |
+------------------------------------+-----+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-------+------+----+-----------------------+------------+----------+------+--------+
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43       |1        |[Churrasqueira, Academia, Playground, Salão de festas, Condomínio fechado, Portão eletrônico, Portaria 24

In [35]:
# contando a quantidade total de linhas do dataset original
count_rows_original = dataset.count()

In [36]:
# juntando os dados
dataset = dataset\
    .join(zona, 'id', how='inner')\
    .join(caracteristicas, 'id', how='left')\
    .drop('zona', 'cara', 'caracteristicas')\
    .na\
    .fill(0)

In [37]:
# comparando se o join funcionou corretamente
# verificando se ainda se mantiveras as mesmas quantidades de linhas
print("Quantidade de linhas antes: ", count_rows_original)
print("Quantidade de linhas depois: ", dataset.count())
print("Quantidade de linhas perdidas: ", count_rows_original - dataset.count())

Quantidade de linhas antes:  66551
Quantidade de linhas depois:  66551
Quantidade de linhas perdidas:  0


In [38]:
# visualizando o Dataset
dataset.show(5)

+--------------------+-----+---------+---------+-------+------+----+-----------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|     bairro|condominio|  iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+-----------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|02fba6ef-a691-442...|    3|       64|        1|      2|   2.0| 1.0|Jacarepaguá|     784.0|  80.0| 380000.0|           0|         0| 

In [39]:
# conferindo o ID e se as colunas dummies referentes a esse ID estão corretas
dataset.filter(f.col('id')=="be269ddf-a4a3-4a72-bd54-93cdccf512dd").show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio|iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|be269ddf-a4a3-4a7...|    0|       45|        1|      2|   0.0| 1.0|Campo Grande|     356.0| 0.0|140000.0|           0|         0|         

## Salvando os dados em parquet
Agora que a base de dados está tratada de forma correta para ser utilizada em um modelo de machine learning, é importante importá-la. Assim será possível utilizar essa base na próxima etapa do projeto.

In [40]:
# salvando os dados no formato parquet
dataset.write.parquet("dados_salvos/dataset_ml_regressao")